In [2]:
!pip install --upgrade pip
!pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1
!pip install \
    transformers==4.27.2 \
    datasets==2.11.0

  Using cached pip-23.2-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Using cached torch-1.13.1-cp37-cp37m-manylinux1_x86_64.whl (887.5 MB)
  Using cached torchdata-0.5.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.6 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached portalocker-2.7.0-py2.py3-none-any.whl (15 kB)
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that the

In [ ]:
!pip install torch==1.13.1


In [3]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [4]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

Found cached dataset csv (/root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-391706c81424fc80/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
example_indices = [40,100]

dash_line = '-'.join("" for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print("EXAMPLE ", i+1)
    print(dash_line)
    print("Human Dialogue:")
    print(dataset["test"][index]["dialogue"])
    print(dash_line)
    print("Human Summary:")
    print(dataset["test"][index]["summary"])

---------------------------------------------------------------------------------------------------
EXAMPLE  1
---------------------------------------------------------------------------------------------------
Human Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Human Summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
EXAMPLE  2
---------------------------------------------------------------------------------------------------
Hu

In [6]:
model_name = "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast = True)

In [8]:
sentence = "What time is it, Tom?"

sentence_encoded = tokenizer(sentence,return_tensors = 'pt')
sentence_decoded = tokenizer.decode(
                    sentence_encoded["input_ids"][0], 
                    skip_special_tokens = True)

print("ENCODED SENTENCE:")
print(sentence_encoded["input_ids"][0])
print("DECODED SENTENCE:")
print(sentence_decoded)

ENCODED SENTENCE:
tensor([ 363,   97,   19,   34,    6, 3059,   58,    1])
DECODED SENTENCE:
What time is it, Tom?


In [10]:
for i,index in enumerate(example_indices):
    dialogue = dataset["test"][index]["dialogue"]
    summary = dataset["test"][index]["summary"]
    
    inputs = tokenizer(dialogue,return_tensors = 'pt')
    
    prediction = model.generate(inputs["input_ids"], max_new_tokens = 50)
    output = tokenizer.decode(prediction[0],skip_special_tokens = True)
    
    print(dash_line)
    print("EXAMPLE ", i+1)
    print(dash_line)
    print("Human Dialogue:")
    print(dialogue)
    print(dash_line)
    print("Machine Summary without prompt engineering:")
    print(output)
    print(dash_line)
    print("Human Summary:")
    print(summary)

---------------------------------------------------------------------------------------------------
EXAMPLE  1
---------------------------------------------------------------------------------------------------
Human Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Machine Summary:
Person1: It's ten to nine.
---------------------------------------------------------------------------------------------------
Human Summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------

In [13]:
for i,index in enumerate(example_indices):
    dialogue = dataset["test"][index]["dialogue"]
    summary = dataset["test"][index]["summary"]
    
    prompt = f"""
    summarize the following conversation.
    {dialogue}
    
    summary:"""
    
    inputs = tokenizer(prompt,return_tensors = 'pt')
    
    prediction = model.generate(inputs["input_ids"], max_new_tokens = 50)
    output = tokenizer.decode(prediction[0],skip_special_tokens = True)
    
    print(dash_line)
    print("EXAMPLE ", i+1)
    print(dash_line)
    print("Human Dialogue:")
    print(prompt)
    print(dash_line)
    print("Machine Summary - Zero Shot:")
    print(output)
    print(dash_line)
    print("Human Summary:")
    print(summary)

---------------------------------------------------------------------------------------------------
EXAMPLE  1
---------------------------------------------------------------------------------------------------
Human Dialogue:

    summarize the following conversation.
    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
    
    summary:
---------------------------------------------------------------------------------------------------
Machine Summary - Zero Shot:
The train is about to leave.
---------------------------------------------------------------------------------------------------
Human Summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty

In [14]:
for i,index in enumerate(example_indices):
    dialogue = dataset["test"][index]["dialogue"]
    summary = dataset["test"][index]["summary"]
    
    prompt = f"""
    A Dialogue follows:
    {dialogue}
    
    What was going on?"""
    
    inputs = tokenizer(prompt,return_tensors = 'pt')
    
    prediction = model.generate(inputs["input_ids"], max_new_tokens = 50)
    output = tokenizer.decode(prediction[0],skip_special_tokens = True)
    
    print(dash_line)
    print("EXAMPLE ", i+1)
    print(dash_line)
    print("Human Dialogue:")
    print(prompt)
    print(dash_line)
    print("Machine Summary - Zero Shot:")
    print(output)
    print(dash_line)
    print("Human Summary:")
    print(summary)

---------------------------------------------------------------------------------------------------
EXAMPLE  1
---------------------------------------------------------------------------------------------------
Human Dialogue:

    A Dialogue follows:
    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
    
    What was going on?
---------------------------------------------------------------------------------------------------
Machine Summary - Zero Shot:
The train is about to leave Tom's place.
---------------------------------------------------------------------------------------------------
Human Summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is pl

In [20]:
def make_prompt(example_indices_full,example_index_to_summarize):
    prompt = ""
    for i,index in enumerate(example_indices_full):
        dialogue = dataset["test"][index]["dialogue"]
        summary = dataset["test"][index]["summary"]

        prompt += f"""
        Dialogue:
        {dialogue}

        What was going on?
        {summary}
        """
        
    dialogue = dataset["test"][example_index_to_summarize]["dialogue"]
        
    prompt += f"""
    Dialogue:
    {dialogue}

    What was going on?
    """
    return prompt

In [16]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full,example_index_to_summarize)
print(one_shot_prompt)


        Dialogue:
        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

        What was going on?
        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
        
        Dialogue:
        #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably nee

In [18]:
summary = dataset["test"][example_index_to_summarize]["summary"]
    
    
    
inputs = tokenizer(one_shot_prompt,return_tensors = 'pt')

prediction = model.generate(inputs["input_ids"], max_new_tokens = 50)
output = tokenizer.decode(prediction[0],skip_special_tokens = True)

print(dash_line)
print("EXAMPLE ")
print(dash_line)
print("Human Dialogue:")
print(one_shot_prompt)
print(dash_line)
print("Machine Summary - One Shot:")
print(output)
print(dash_line)
print("Human Summary:")
print(summary)

---------------------------------------------------------------------------------------------------
EXAMPLE 
---------------------------------------------------------------------------------------------------
Human Dialogue:

        Dialogue:
        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

        What was going on?
        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
        
        Dialogue:
        #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyer

In [21]:
example_indices_full = [40, 80,120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full,example_index_to_summarize)
print(few_shot_prompt)


        Dialogue:
        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

        What was going on?
        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
        
        Dialogue:
        #Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable p

In [22]:
summary = dataset["test"][example_index_to_summarize]["summary"]
    
    
    
inputs = tokenizer(few_shot_prompt,return_tensors = 'pt')

prediction = model.generate(inputs["input_ids"], max_new_tokens = 50)
output = tokenizer.decode(prediction[0],skip_special_tokens = True)

print(dash_line)
print("EXAMPLE ")
print(dash_line)
print("Human Dialogue:")
print(few_shot_prompt)
print(dash_line)
print("Machine Summary - One Shot:")
print(output)
print(dash_line)
print("Human Summary:")
print(summary)

Token indices sequence length is longer than the specified maximum sequence length for this model (819 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
EXAMPLE 
---------------------------------------------------------------------------------------------------
Human Dialogue:

        Dialogue:
        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

        What was going on?
        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
        
        Dialogue:
        #Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. S

In [23]:
generation_config = GenerationConfig(max_new_tokens = 50)
#generation_config = GenerationConfig(max_new_tokens = 50,do_sample = True, temperature = 0.1)

summary = dataset["test"][example_index_to_summarize]["summary"]
    
    
    
inputs = tokenizer(few_shot_prompt,return_tensors = 'pt')

prediction = model.generate(inputs["input_ids"], generation_config = generation_config)
output = tokenizer.decode(prediction[0],skip_special_tokens = True)

print(dash_line)
print("EXAMPLE ")
print(dash_line)
print("Human Dialogue:")
print(few_shot_prompt)
print(dash_line)
print("Machine Summary - One Shot:")
print(output)
print(dash_line)
print("Human Summary:")
print(summary)

---------------------------------------------------------------------------------------------------
EXAMPLE 
---------------------------------------------------------------------------------------------------
Human Dialogue:

        Dialogue:
        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

        What was going on?
        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
        
        Dialogue:
        #Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. S